# Sticky Session Routing

This notebook demonstrates how to use sticky session routing with vLLM on SageMaker.

## Overview

Sticky session routing enables all requests from the same session to be routed to the same model instance, allowing your application to reuse previously processed information.

### How It Works
1. Enable sessions with `SAGEMAKER_ENABLE_STATEFUL_SESSIONS=true`
2. Create a session with `SessionId="NEW_SESSION"`
3. Use the returned session ID for subsequent requests
4. Close the session when done to free resources

## Prerequisites
- SageMaker execution role with appropriate permissions
- vLLM container image in ECR
- HuggingFace token for gated models (e.g., Llama)


In [3]:
import boto3
import json
import time
from datetime import datetime

In [ ]:
session = boto3.Session()
region = session.region_name
sagemaker_client = boto3.client('sagemaker', region_name=region)
runtime_client = boto3.client('sagemaker-runtime', region_name=region)
sts_client = boto3.client('sts', region_name=region)

In [ ]:
timestamp = datetime.now().strftime('%Y%m%d-%H%M%S')
model_name = f'vllm-sticky-session-{timestamp}'
endpoint_config_name = f'vllm-sticky-session-config-{timestamp}'
endpoint_name = f'vllm-sticky-session-{timestamp}'
account_id = sts_client.get_caller_identity()['Account']

## 1. Create Model with Sticky Sessions Enabled

In [ ]:
container_image = f'{account_id}.dkr.ecr.{region}.amazonaws.com/vllm:0.11.2-sagemaker-v1.2'
huggingface_model_id = 'meta-llama/Meta-Llama-3-8B-Instruct'
huggingface_token = ''  # Replace with your actual token
instance_type = 'ml.g6.4xlarge'
execution_role = f'arn:aws:iam::{account_id}:role/SageMakerExecutionRole'

In [ ]:
print("Configuration:")
print(f"  Model Name: {model_name}")
print(f"  Endpoint Name: {endpoint_name}")
print(f"  HuggingFace Model: {huggingface_model_id}")
print(f"  Instance Type: {instance_type}")

Configuration:
  Model Name: vllm-sticky-session-20251211-225002
  Endpoint Name: vllm-sticky-session-20251211-225002
  HuggingFace Model: meta-llama/Meta-Llama-3-8B-Instruct
  Instance Type: ml.g6.4xlarge


In [ ]:
create_model_response = sagemaker_client.create_model(
    ModelName=model_name,
    PrimaryContainer={
        'Image': container_image,
        'Environment': {
            'SM_VLLM_MODEL': huggingface_model_id,
            'HUGGING_FACE_HUB_TOKEN': huggingface_token,
            'SAGEMAKER_CONTAINER_LOG_LEVEL': 'INFO',
            'SM_VLLM_MAX_MODEL_LEN': '2048',
            # Enable sticky session routing
            'SAGEMAKER_ENABLE_STATEFUL_SESSIONS': 'true',
        }
    },
    ExecutionRoleArn=execution_role,
)
print(f"✓ Model created")
print(f"  Model ARN: {create_model_response['ModelArn']}")

✓ Model created
  Model ARN: arn:aws:sagemaker:us-west-2:875423407011:model/vllm-sticky-session-20251211-225002


## 2. Create Endpoint Configuration

In [ ]:
print(f"\nCreating endpoint configuration: {endpoint_config_name}")

# Note: Use 2+ instances to see sticky routing in action
# With 1 instance, all requests go to the same instance anyway
instance_count = 2

create_endpoint_config_response = sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            'VariantName': 'AllTraffic',
            'ModelName': model_name,
            'InstanceType': instance_type,
            'InitialInstanceCount': instance_count,
            'InitialVariantWeight': 1.0,
        }
    ]
)

print(f"✓ Endpoint configuration created")
print(f"  Config ARN: {create_endpoint_config_response['EndpointConfigArn']}")

## 3. Create Endpoint

In [ ]:
print(f"\nCreating endpoint: {endpoint_name}")
print("⏱️  This will take approximately 5-10 minutes...")
print(f"\n💡 Monitor progress: https://console.aws.amazon.com/sagemaker/home?region={region}#/endpoints/{endpoint_name}\n")

create_endpoint_response = sagemaker_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name
)

print(f"✓ Endpoint creation initiated")
print(f"  Endpoint ARN: {create_endpoint_response['EndpointArn']}")



Creating endpoint: vllm-sticky-session-20251211-225002
⏱️  This will take approximately 5-10 minutes...

💡 Monitor progress: https://console.aws.amazon.com/sagemaker/home?region=us-west-2#/endpoints/vllm-sticky-session-20251211-225002

✓ Endpoint creation initiated
  Endpoint ARN: arn:aws:sagemaker:us-west-2:875423407011:endpoint/vllm-sticky-session-20251211-225002


In [ ]:
print("\nWaiting for endpoint to be in service...")
print("(This may take 5-10 minutes - please be patient)\n")

waiter = sagemaker_client.get_waiter('endpoint_in_service')
waiter.wait(
    EndpointName=endpoint_name,
    WaiterConfig={
        'Delay': 20,
        'MaxAttempts': 60
    }
)
print("✓ Endpoint is now in service!")


Waiting for endpoint to be in service...
(This may take 5-10 minutes - please be patient)

✓ Endpoint is now in service!


## 4. Create a Session

In [ ]:
response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/json',
    SessionId="NEW_SESSION",
    Body=json.dumps({"requestType": "NEW_SESSION"})
)

In [ ]:
def extract_session_id_from_header(header_value: str) -> str:
    """Extract session ID from SageMaker session header.

    Header format: "<uuid>; Expires=<timestamp>"
    """
    # The session ID is before the semicolon
    if ";" in header_value:
        return header_value.split(";")[0].strip()
    return header_value.strip()


In [ ]:
# Get session ID from response header
header_value = response['ResponseMetadata']['HTTPHeaders']['x-amzn-sagemaker-new-session-id']
session_id = extract_session_id_from_header(header_value)
print(f"✓ Session created")
print(f"  Session ID: {session_id}")

## 5. Use the Session (Chat Completions)
 
Using OpenAI-compatible chat completions format. With sticky sessions,
the KV cache from previous turns can be reused for faster inference.


In [ ]:
# Initialize conversation history
messages = []

# First message - introduce yourself
messages.append({"role": "user", "content": "Hi! My name is Alice and I love hiking."})

response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/json',
    SessionId=session_id,
    Body=json.dumps({
        "model": huggingface_model_id,
        "messages": messages,
        "max_tokens": 100,
        "temperature": 0.7
    })
)

In [ ]:

result = json.loads(response['Body'].read())
assistant_message = result['choices'][0]['message']['content']
messages.append({"role": "assistant", "content": assistant_message})

print("User: Hi! My name is Alice and I love hiking.")
print(f"Assistant: {assistant_message}")
print(f"\nUsage: {result.get('usage', {})}")



User: Hi! My name is Alice and I love hiking.
Assistant: Hi Alice! It's great to meet you! I'm glad to hear that you love hiking. There's something about being in nature, surrounded by trees, mountains, and wildlife, that can be so uplifting and rejuvenating. What kind of hiking do you enjoy the most? Do you have a favorite trail or location that you like to hike?

Usage: {'prompt_tokens': 21, 'total_tokens': 92, 'completion_tokens': 71, 'prompt_tokens_details': None}


In [ ]:
# Second message - ask about previous context (tests KV cache reuse)
messages.append({"role": "user", "content": "What's my name and what do I enjoy doing?"})

response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/json',
    SessionId=session_id,
    Body=json.dumps({
        "model": huggingface_model_id,
        "messages": messages,
        "max_tokens": 100,
        "temperature": 0.7
    })
)

result = json.loads(response['Body'].read())
assistant_message = result['choices'][0]['message']['content']
messages.append({"role": "assistant", "content": assistant_message})

print("User: What's my name and what do I enjoy doing?")
print(f"Assistant: {assistant_message}")
print(f"\nUsage: {result.get('usage', {})}")

User: What's my name and what do I enjoy doing?
Assistant: Your name is Alice, and you enjoy hiking!

Usage: {'prompt_tokens': 112, 'total_tokens': 123, 'completion_tokens': 11, 'prompt_tokens_details': None}


In [ ]:
# Third message - continue the conversation
messages.append({"role": "user", "content": "Can you suggest a good hiking trail?"})

response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/json',
    SessionId=session_id,
    Body=json.dumps({
        "model": huggingface_model_id,
        "messages": messages,
        "max_tokens": 150,
        "temperature": 0.7
    })
)

result = json.loads(response['Body'].read())
assistant_message = result['choices'][0]['message']['content']

print("User: Can you suggest a good hiking trail?")
print(f"Assistant: {assistant_message}")
print(f"\nUsage: {result.get('usage', {})}")

User: Can you suggest a good hiking trail?
Assistant: I'd be happy to suggest a good hiking trail!

Since you didn't specify a location or a level of difficulty, I'll suggest a few popular trails that are suitable for most hikers:

1. **Appalachian Trail, USA**: The Appalachian Trail is a 2,190-mile long trail that spans from Georgia to Maine. It's a challenging trail, but it's also incredibly scenic and offers stunning views of the Appalachian Mountains.
2. **Inca Trail, Peru**: The Inca Trail is a 26-mile long trail that leads to Machu Picchu, one of the Seven Wonders of the World. It's a challenging hike, but the scenery is breathtaking, and the history is rich.
3. **Troll

Usage: {'prompt_tokens': 140, 'total_tokens': 290, 'completion_tokens': 150, 'prompt_tokens_details': None}


## Verification

In the cloudwatch log, We can see that the session is created and all conversation is routed to the same instance. The Prefix cache is also utilized for the same conversation.

```
#033[1;36m(APIServer pid=1)#033[0;0m INFO:     169.254.178.2:40944 - "GET /ping HTTP/1.1" 200 OK
#033[1;36m(APIServer pid=1)#033[0;0m [INFO] model_hosting_container_standards - handlers.py:80: Session 99680e4c-5d0e-49ac-88e7-103866569520 created
#033[1;36m(APIServer pid=1)#033[0;0m INFO:     169.254.178.2:40944 - "POST /invocations HTTP/1.1" 200 OK
#033[1;36m(APIServer pid=1)#033[0;0m INFO:     169.254.178.2:40944 - "GET /ping HTTP/1.1" 200 OK
#033[1;36m(APIServer pid=1)#033[0;0m INFO:     169.254.178.2:37650 - "GET /ping HTTP/1.1" 200 OK
#033[1;36m(APIServer pid=1)#033[0;0m INFO 12-11 16:05:14 [loggers.py:236] Engine 000: Avg prompt throughput: 2.1 tokens/s, Avg generation throughput: 1.7 tokens/s, Running: 1 reqs, Waiting: 0 reqs, GPU KV cache usage: 0.2%, Prefix cache hit rate: 79.4%
#033[1;36m(APIServer pid=1)#033[0;0m INFO:     169.254.178.2:40944 - "POST /invocations HTTP/1.1" 200 OK
#033[1;36m(APIServer pid=1)#033[0;0m INFO:     169.254.178.2:40944 - "POST /invocations HTTP/1.1" 200 OK
#033[1;36m(APIServer pid=1)#033[0;0m INFO:     169.254.178.2:37650 - "GET /ping HTTP/1.1" 200 OK
#033[1;36m(APIServer pid=1)#033[0;0m INFO:     169.254.178.2:37650 - "GET /ping HTTP/1.1" 200 OK
#033[1;36m(APIServer pid=1)#033[0;0m INFO 12-11 16:05:24 [loggers.py:236] Engine 000: Avg prompt throughput: 25.2 tokens/s, Avg generation throughput: 16.6 tokens/s, Running: 1 reqs, Waiting: 0 reqs, GPU KV cache usage: 0.8%, Prefix cache hit rate: 79.0%
#033[1;36m(APIServer pid=1)#033[0;0m INFO:     169.254.178.2:40944 - "POST /invocations HTTP/1.1" 200 OK
#033[1;36m(APIServer pid=1)#033[0;0m INFO:     169.254.178.2:40944 - "GET /ping HTTP/1.1" 200 OK
#033[1;36m(APIServer pid=1)#033[0;0m INFO:     169.254.178.2:40944 - "GET /ping HTTP/1.1" 200 OK
#033[1;36m(APIServer pid=1)#033[0;0m INFO 12-11 16:05:34 [loggers.py:236] Engine 000: Avg prompt throughput: 0.0 tokens/s, Avg generation throughput: 4.9 tokens/s, Running: 0 reqs, Waiting: 0 reqs, GPU KV cache usage: 0.0%, Prefix cache hit rate: 79.0%
#033[1;36m(APIServer pid=1)#033[0;0m INFO:     169.254.178.2:40944 - "GET /ping HTTP/1.1" 200 OK
#033[1;36m(APIServer pid=1)#033[0;0m INFO:     169.254.178.2:40944 - "GET /ping HTTP/1.1" 200 OK
#033[1;36m(APIServer pid=1)#033[0;0m INFO 12-11 16:05:44 [loggers.py:236] Engine 000: Avg prompt throughput: 0.0 tokens/s, Avg generation throughput: 0.0 tokens/s, Running: 0 reqs, Waiting: 0 reqs, GPU KV cache usage: 0.0%, Prefix cache hit rate: 79.0%
```

## 6. Close the Session

In [ ]:
# Close the session when done to free resources
response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/json',
    SessionId=session_id,
    Body=json.dumps({"requestType": "CLOSE"})
)
print(response)
print("✓ Session closed")

{'ResponseMetadata': {'RequestId': 'f39176fa-4057-45c7-b215-521b79b45260', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'f39176fa-4057-45c7-b215-521b79b45260', 'x-amzn-sagemaker-closed-session-id': '99680e4c-5d0e-49ac-88e7-103866569520', 'x-amzn-invoked-production-variant': 'AllTraffic', 'date': 'Fri, 12 Dec 2025 00:17:01 GMT', 'content-type': 'application/json', 'content-length': '51', 'connection': 'keep-alive'}, 'RetryAttempts': 0}, 'ContentType': 'application/json', 'InvokedProductionVariant': 'AllTraffic', 'ClosedSessionId': '99680e4c-5d0e-49ac-88e7-103866569520', 'Body': <botocore.response.StreamingBody object at 0x7f66236b6da0>}
✓ Session closed


## 7. Cleanup

In [ ]:
# Delete endpoint
sagemaker_client.delete_endpoint(EndpointName=endpoint_name)
sagemaker_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sagemaker_client.delete_model(ModelName=model_name)
print("✓ Resources cleaned up")

✓ Resources cleaned up
